In [21]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn

In [10]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms =  transforms.Compose([
        transforms.RandomSizedCrop(30),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])


image_dataset = datasets.ImageFolder("./patches", transform=data_transforms)
dataloders = torch.utils.data.DataLoader(image_dataset,
                                         batch_size=16,
                                         shuffle=True,
                                         num_workers=4)


class_names = image_dataset.classes

use_gpu = torch.cuda.is_available()

In [31]:
help(nn.MaxPool2d)

Help on class MaxPool2d in module torch.nn.modules.pooling:

class MaxPool2d(torch.nn.modules.module.Module)
 |  Applies a 2D max pooling over an input signal composed of several input
 |  planes.
 |  
 |  In the simplest case, the output value of the layer with input size :math:`(N, C, H, W)`,
 |  output :math:`(N, C, H_{out}, W_{out})` and :attr:`kernel_size` :math:`(kH, kW)`
 |  can be precisely described as:
 |  
 |  .. math::
 |  
 |      \begin{array}{ll}
 |      out(N_i, C_j, h, w)  = \max_{{m}=0}^{kH-1} \max_{{n}=0}^{kW-1}
 |                             input(N_i, C_j, stride[0] * h + m, stride[1] * w + n)
 |      \end{array}
 |  
 |  | If :attr:`padding` is non-zero, then the input is implicitly zero-padded on both sides
 |    for :attr:`padding` number of points
 |  | :attr:`dilation` controls the spacing between the kernel points. It is harder to describe,
 |    but this `link`_ has a nice visualization of what :attr:`dilation` does.
 |  
 |  The parameters :attr:`kernel_siz

In [39]:
class ConvNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(ConvNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 64 * 4 * 4)
        x = self.classifier(x)
        return x

model = ConvNet(num_classes=10)

model.features = torch.nn.DataParallel(model.features)
model.classifier = torch.nn.DataParallel(model.classifier)
model.cuda()

ConvNet (
  (features): DataParallel (
    (module): Sequential (
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
      (1): ReLU (inplace)
      (2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
      (4): ReLU (inplace)
      (5): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
      (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU (inplace)
      (8): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    )
  )
  (classifier): DataParallel (
    (module): Sequential (
      (0): Dropout (p = 0.5)
      (1): Linear (1024 -> 256)
      (2): ReLU (inplace)
      (3): Dropout (p = 0.5)
      (4): Linear (256 -> 256)
      (5): ReLU (inplace)
      (6): Linear (256 -> 10)
    )
  )
)

In [37]:
# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), 0.001,
                            momentum=False,
                            )

In [ ]:
# switch to train mode
model.train()

for epoch in range(100):
    for i, (input, target) in enumerate(dataloders):

        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
    

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if (i%1000) == 0:
            print('Epoch: [[ {0} ]] [{1}/{2}]\t'.format(epoch, i, len(dataloders)))

Epoch: [[ 0 ]] [0/2173]	
Epoch: [[ 0 ]] [1000/2173]	
Epoch: [[ 0 ]] [2000/2173]	
Epoch: [[ 1 ]] [0/2173]	
Epoch: [[ 1 ]] [1000/2173]	
Epoch: [[ 1 ]] [2000/2173]	
Epoch: [[ 2 ]] [0/2173]	
Epoch: [[ 2 ]] [1000/2173]	
Epoch: [[ 2 ]] [2000/2173]	
Epoch: [[ 3 ]] [0/2173]	
Epoch: [[ 3 ]] [1000/2173]	
Epoch: [[ 3 ]] [2000/2173]	
Epoch: [[ 4 ]] [0/2173]	
Epoch: [[ 4 ]] [1000/2173]	
Epoch: [[ 4 ]] [2000/2173]	
Epoch: [[ 5 ]] [0/2173]	
Epoch: [[ 5 ]] [1000/2173]	
Epoch: [[ 5 ]] [2000/2173]	
Epoch: [[ 6 ]] [0/2173]	
Epoch: [[ 6 ]] [1000/2173]	
Epoch: [[ 6 ]] [2000/2173]	
Epoch: [[ 7 ]] [0/2173]	
Epoch: [[ 7 ]] [1000/2173]	


In [67]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

# switch to evaluate mode
model.eval()

losses = AverageMeter()
top1 = AverageMeter()
top5 = AverageMeter()


for i, (input, target) in enumerate(dataloders):
    target = target.cuda(async=True)
    input_var = torch.autograd.Variable(input, volatile=True)
    target_var = torch.autograd.Variable(target, volatile=True)

    # compute output
    output = model(input_var)
    loss = criterion(output, target_var)

    # measure accuracy and record loss
    prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
    
    losses.update(loss.data[0], input.size(0))
    top1.update(prec1[0], input.size(0))
    top5.update(prec5[0], input.size(0))

    if i % 100 == 0:
        print('Test: [{0}/{1}]\t'.format(i, len(dataloders)))

#print('* Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))

Test: [0/2173]	
Test: [100/2173]	
Test: [200/2173]	
Test: [300/2173]	
Test: [400/2173]	
Test: [500/2173]	
Test: [600/2173]	
Test: [700/2173]	
Test: [800/2173]	
Test: [900/2173]	
Test: [1000/2173]	
Test: [1100/2173]	
Test: [1200/2173]	
Test: [1300/2173]	
Test: [1400/2173]	
Test: [1500/2173]	
Test: [1600/2173]	
Test: [1700/2173]	
Test: [1800/2173]	
Test: [1900/2173]	
Test: [2000/2173]	
Test: [2100/2173]	


In [65]:

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [73]:
top5.avg

45.54478234600224